In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!unzip "/content/drive/MyDrive/Parrot/img.zip" -d "./"

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import glob

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.utils import io

from packaging import version

import logging
logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [ ]:
import tensorflow as tf
print("TensorFlow version: {}".format(tf.__version__))
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

%load_ext tensorboard
from tensorboard.plugins.hparams import api as hp

gpu_list = tf.config.list_physical_devices('GPU')
print("GPUs Available:")
if gpu_list:
    try:
        for gpu in gpu_list:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpu_list), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
else:
    print('None')


tf.keras.backend.clear_session()

from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

debug_mode = False
tf.config.run_functions_eagerly(debug_mode)
if debug_mode:
    tf.data.experimental.enable_debug_mode()

print("Eager execution: {}".format(tf.executing_eagerly()))

In [ ]:
batch_size = 32

data_path = './img/'
train_ds = tf.data.Dataset.list_files(os.path.join(data_path, 'train', '*.png'))
test_ds = tf.data.Dataset.list_files(os.path.join(data_path, 'test', '*.png'))

def process_path(file_path):
    label = tf.strings.to_number(tf.strings.split(file_path, '_')[1])
    image = tf.io.decode_png(tf.io.read_file(file_path))
    return image, label

train_ds = train_ds.map(process_path,
                         num_parallel_calls=tf.data.experimental.AUTOTUNE,
                         deterministic=False
                        ).batch(batch_size,
                                num_parallel_calls=tf.data.experimental.AUTOTUNE,
                                deterministic=False).prefetch(tf.data.experimental.AUTOTUNE).cache()

test_ds = test_ds.map(process_path,
                         num_parallel_calls=tf.data.experimental.AUTOTUNE,
                         deterministic=True
                        ).batch(batch_size,
                                num_parallel_calls=tf.data.experimental.AUTOTUNE,
                                deterministic=True).prefetch(tf.data.experimental.AUTOTUNE).cache()

In [ ]:
img_height = 9
img_width = 9
channels = 3
num_classes = 2

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, channels)),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.BatchNormalization(axis=-1),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.BatchNormalization(axis=-1),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  tf.keras.layers.BatchNormalization(axis=-1),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

optimizer = mixed_precision.LossScaleOptimizer(tf.keras.optimizers.Adam(learning_rate=1e-5))
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric_fn = tf.keras.metrics.Accuracy()

epochs = 100

log_dir = './tf_logs/'
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_metric = tf.keras.metrics.Mean('test_metric', dtype=tf.float32)
summary_writer_train = tf.summary.create_file_writer(os.path.join(log_dir, 'Train'))
summary_writer_test = tf.summary.create_file_writer(os.path.join(log_dir, 'Test'))
progression_bar = tf.keras.utils.Progbar(target=epochs, verbose=1, unit_name='epoch',
                                                 stateful_metrics=['Train Loss'])

@tf.function()
def _train_step(_x_train, _y_train):
    with tf.GradientTape() as tape:
        predictions = model(_x_train, training=True)
        loss = loss_fn(_y_train, predictions)
        scaled_loss = optimizer.get_scaled_loss(loss)
    scaled_gradients = tape.gradient(scaled_loss, model.trainable_variables)
    gradients = optimizer.get_unscaled_gradients(scaled_gradients)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)

@tf.function()
def _test_step(_x_test, _y_test):
    predictions = model(_x_test, training=False)
    loss = loss_fn(_y_test, predictions)
    test_loss(loss)
    #metric = metric_fn(_y_test, predictions)
    #test_metric(metric)

for epoch in range(epochs):
    tf.summary.experimental.set_step(epoch)
    for (x_train, y_train) in train_ds:
      _train_step(x_train, y_train)
    for (x_test, y_test) in test_ds:
         _test_step(x_test, y_test)
    with summary_writer_train.as_default():
        tf.summary.scalar('Loss', train_loss.result(), step=epoch)
    with summary_writer_test.as_default():
        tf.summary.scalar('Loss', test_loss.result(), step=epoch)
    progression_bar.update(current=epoch + 1,
                           values=[('Train Loss', train_loss.result()),
                                   ('Test Loss', test_loss.result())])
    train_loss.reset_states()

